# Import txt output from Buehler code

In [1]:
!pip install xlsxwriter

In [2]:
import os 
import glob
import pandas as pd

# Specify the directory
current_path = os.getcwd()
directory_path = current_path + '\\buehler_output'

# Use glob to find all .txt files in the directory
txt_files = glob.glob(os.path.join(directory_path, '*.txt'))

In [8]:
identifiers = [
    "0,2", 
    "10,0",
    "25,0"
]
with pd.ExcelWriter(current_path + '/metrics.xlsx', engine='xlsxwriter') as writer:
    for identifier in identifiers:
        # Initialize dictionaries to store metrics
        metric1_data = {}
        metric2_data = {}
        count = 0
        # Loop through the list of .txt files
        for txt_file in txt_files:
            specification = txt_file.split('\\')[-1].split("_")[1:]
            if specification[-2] != identifier:
                continue
            if specification[0] == 'GBM':
                start_index = 0
            elif specification[0] == 'Kou':
                start_index = 2
            else: 
                continue
            count += 1
            # Open and read the file
            with open(txt_file, 'r') as file:
                content = file.read()
                metric_strings = content.split('\n')

                n_in = int(specification[1+start_index])
                n_out = int(specification[2+start_index])
                metric1 = metric_strings[0].split('=')[1][1:-1]
                metric1 = round(float(metric1)/100,6)
                metric2 = metric_strings[1].split('=')[1][1:-1]
                metric2 = round(float(metric2)/100,6)
                
                # Add data to dictionaries
                if n_in not in metric1_data:
                    metric1_data[n_in] = {}
                if n_in not in metric2_data:
                    metric2_data[n_in] = {}
                    
                metric1_data[n_in][n_out] = metric1
                metric2_data[n_in][n_out] = metric2

                if count == 25:
                    # Create DataFrames from the dictionaries
                    metric1_df = pd.DataFrame.from_dict(metric1_data, orient='index').sort_index(axis=0).sort_index(axis=1)
                    metric2_df = pd.DataFrame.from_dict(metric2_data, orient='index').sort_index(axis=0).sort_index(axis=1)
                    print(metric1_data)
                    print(metric1_df)
                    # Export the DataFrames to different sheets in the same Excel file
                    metric1_df.to_excel(writer, sheet_name=f'metric1_{specification[0]+identifier}', index=True)
                    metric2_df.to_excel(writer, sheet_name=f'metric2_{specification[0]+identifier}', index=True)
                    # print(metric1_df)
                    # print(metric2_df)
                    # Reset dictionaries to store metrics
                    metric1_data = {}
                    metric2_data = {}
                    count = 0

{100000: {100000: 0.917912, 10000: 0.999921, 1000: 0.999777, 100: 0.998949, 10: 0.999922}, 10000: {100000: 0.966019, 10000: 0.999846, 1000: 0.999957, 100: 0.999422, 10: 0.998781}, 1000: {100000: 0.990544, 10000: 0.99971, 1000: 0.999828, 100: 0.999293, 10: 0.989098}, 100: {100000: 0.945937, 10000: 0.936027, 1000: 0.995253, 100: 0.99299, 10: 0.999084}, 10: {100000: 0.0, 10000: 0.0, 1000: 0.000215, 100: 0.000992, 10: 0.108333}}
          10        100       1000      10000     100000
10      0.108333  0.000992  0.000215  0.000000  0.000000
100     0.999084  0.992990  0.995253  0.936027  0.945937
1000    0.989098  0.999293  0.999828  0.999710  0.990544
10000   0.998781  0.999422  0.999957  0.999846  0.966019
100000  0.999922  0.998949  0.999777  0.999921  0.917912
{100000: {100000: 0.994339, 10000: 0.999906, 1000: 0.999868, 100: 0.999995, 10: 0.99943}, 10000: {100000: 0.999295, 10000: 0.999817, 1000: 0.99998, 100: 0.999967, 10: 0.999841}, 1000: {100000: 0.939121, 10000: 0.998498, 1000: 0.9